In [15]:
import simpy
import random
import numpy
import pandas as pd

env = simpy.Environment()

cashier_count = 10
barista_count = 20
customer_count = 1000
shop_uptime = 15 * 60

cashier = simpy.Resource(env, capacity=cashier_count)
barista = simpy.Resource(env, capacity=barista_count)

menu = {
    1: ["Regular Coffee", 10, 15],
    2: ["Latte", 30, 45],
    3: ["Mocha", 30, 45],
    4: ["Cold Brew", 10, 20],
    5: ["Frappe", 50, 70],
    6: ["Espresso", 20, 35],
    7: ["Coffee of the Day", 60, 80]
}

payment_options = {
    1: ["Cash", 15, 30],
    2: ["Card", 10, 20],
    3: ["Online Payment", 5, 15]
}

payment_wait_time = []
payment_time = []
order_wait_time = []
order_time = []

customer_data = []


def generate_customer(env, cashier, barista):
    while env.now < shop_uptime:
        yield env.timeout(random.randint(1, 20))
        env.process(customer(env, env.now, cashier, barista))



def customer(env, name, cashier, barista):
    print("Customer %s arrived at time %.1f" % (name, env.now))

    with cashier.request() as req:
        start_cq = env.now
        yield req
        payment_wait_time.append(env.now - start_cq)

        menu_item = random.randint(1, 6)
        payment_type = random.randint(1, 2)
        time_to_order = random.randint(payment_options[payment_type][1], payment_options[payment_type][2])
        payment_name = payment_options[payment_type][0]

        yield env.timeout(time_to_order)
        print(">>> Customer %s finished paying by %s in %.1f seconds" % (name, payment_name, env.now - start_cq))
        payment_time.append(env.now - start_cq)

    with barista.request() as req:
        start_bq = env.now
        yield req
        order_wait_time.append(env.now - start_bq)

        time_to_prepare = random.randint(menu[menu_item][1], menu[menu_item][2])
        item_name = menu[menu_item][0]

        yield env.timeout(time_to_prepare)
        print(">> >> >> Customer %s served %s in %.1f seconds" % (name, item_name, env.now - start_cq))
        order_time.append(env.now - start_cq)

    customer_data.append((
        payment_wait_time[-1],  # Last added payment wait time
        payment_time[-1],       # Last added payment time
        order_wait_time[-1],    # Last added order wait time
        order_time[-1]          # Last added order time
    ))


env.process(generate_customer(env, cashier, barista))

print("WITH %s CASHIERS AND %s BARISTAS AND %s SERIALLY ARRIVING CUSTOMERS..." % (cashier_count, barista_count, customer_count))

env.run(until=shop_uptime)

WITH 10 CASHIERS AND 20 BARISTAS AND 1000 SERIALLY ARRIVING CUSTOMERS...
Customer 11 arrived at time 11.0
Customer 27 arrived at time 27.0
>>> Customer 11 finished paying by Cash in 20.0 seconds
Customer 31 arrived at time 31.0
Customer 44 arrived at time 44.0
>>> Customer 27 finished paying by Card in 19.0 seconds
>>> Customer 31 finished paying by Cash in 16.0 seconds
>> >> >> Customer 11 served Cold Brew in 38.0 seconds
Customer 56 arrived at time 56.0
Customer 65 arrived at time 65.0
>>> Customer 44 finished paying by Cash in 27.0 seconds
>>> Customer 56 finished paying by Cash in 16.0 seconds
>> >> >> Customer 31 served Mocha in 52.0 seconds
Customer 85 arrived at time 85.0
>>> Customer 65 finished paying by Card in 20.0 seconds
>> >> >> Customer 56 served Cold Brew in 29.0 seconds
>> >> >> Customer 27 served Mocha in 63.0 seconds
Customer 97 arrived at time 97.0
>>> Customer 85 finished paying by Card in 19.0 seconds
>> >> >> Customer 44 served Espresso in 61.0 seconds
Customer 1

In [16]:
print("Average wait time in payment queue: %.1f seconds." % (numpy.mean(payment_wait_time)))
print("Average time until making the payment: %.1f seconds." % (numpy.mean(payment_time)))
print("Average wait time in order queue: %.1f seconds." % (numpy.mean(order_wait_time)))
print("Average time until order is serviced: %.1f seconds." % (numpy.mean(order_time)))
print("Total simulation time: %.1f minutes." % env.now)

Average wait time in payment queue: 0.0 seconds.
Average time until making the payment: 18.2 seconds.
Average wait time in order queue: 0.0 seconds.
Average time until order is serviced: 46.2 seconds.
Total simulation time: 900.0 minutes.


In [ ]:
df = pd.DataFrame(customer_data, columns=["Payment Wait Time", "Payment Time", "Order Wait Time", "Order Time"])

# Display first 10 rows
df.head(10)

,Payment Wait Time,Payment Time,Order Wait Time,Order Time
0,0,16,0,38
1,0,16,0,52
2,0,20,0,29
3,0,20,0,63
4,0,19,0,61
5,0,13,0,35
6,0,13,0,25
7,0,13,0,71
8,0,13,0,28
9,0,17,0,23
